In [73]:
import os
import json
from IPython.display import HTML, display
try:
    (notebook_dir, e)
except:
    e = "a --- b"
    notebook_dir = "."

In [74]:
padding = {
    "space": 275,  # Standard space width
    "i_ia": 80,
    "s_": -80,
    "ks_": -180,
    "ts_": -380,
    "kH": -280,
    "kL": -390,
    "ta_": -250,
    "to_": -250,
    "tb_": -250,
    "tc_": -280,
    "tuR_": -340,
}

### Nommage des fichiers : 
* "ta_" suffit si la lettre attendue est \_a; a_ attend un a à gauche (ou une lettre compatible), pareil dans l'autre sens.
* uL et uR (Left; Right par rapp à la direction vers laquelle l'arche pointe); bL et bR (direction de la pointe pour les heads) ; les autres lesttres sont en a_left; a_right; etc
* Les autres glyphes sont notés comme leur composante principale (par ex "ks") puis un commentaire : ks_above_t

### TODO :
* Découper uL et uR en deux formes, avec des connecteurs "avant" et "après" pour interfacer avec fH et n. Revoir le design de la lettre... Créer les jonctions (t, s) en un glyph bien défini.
* Ne reste que les ligatures spéciales pour les voyelles précédent les clusters, les formes T basses, et "i" "l" "h" 
* Implémenter les mécaniques "d" et "g"

In [75]:
fea = '''

table GDEF {
    GlyphClassDef
        [], # Base glyphs
        [], # Ligature glyphs
        [i j h h_ha h_ah i_aj i_ja i_tia], # Mark glyphs
        []; # Component glyphs
} GDEF;

feature ccmp {
    script DFLT;
    language dflt required;
    script latn;
    language dflt required;

    # ======================================================== 1 LETTRES FLOTTANTES
    
    lookup ETAPE_1_FLOTTANTES {  # Décide si les lettres flottantes doivent devenir des diacritiques ou non

        sub i' [t] by i_it;
        sub i' [s] by i_sia;
        sub [t] i' [a o e p y bL bR uL uR] by i_tia;

        sub [a o e p y bL bR uL uR] i' [a o e p y bL bR uL uR] by i_aj;
        sub i' by i_ia;
        sub j' by i_ja;

        sub h' [a o e p y bL bR uL uR] by h_ha;
        sub [a o e p y bL bR uL uR] h' by h_ah;
        
        # TODO : Group diacritics together into a group. They will be positioned later depending on the surounding letters.

    } ETAPE_1_FLOTTANTES;

    # ======================================================== 2 CHOIX CONTEXTUEL

    lookup ETAPE_2_CHOIX {  # Choix forme contextuelle de base
        
        lookupflag IgnoreMarks;

        # Lettre F

        sub [a p] f' by fH;
        sub [o e] f' by fL;

        sub f' [a c u] by fH;
        sub k' [a c u b] by kH;

        sub f' [o b] by fL;  # This isnt optimal but it's easier to read
        sub k' [o b] by kL;

        # Lettre B

        sub [o n fL] b' by bR;
        sub b' by bL;

        # Lettre U

        sub [t s] u' by uR;  # Ignore next letter
        sub [n] u' by uL;  
        sub u' [a e p o b c y n] by uR;
        sub u' [f] [a e p c u n] by uR;
        sub u' by uL;

    } ETAPE_2_CHOIX;

    # ======================================================== 3 COMPOSITION FUSION
    
    lookup ETAPE_3_COMPOSANTS {  # Decoupe tous les caracteres en composants

        lookupflag IgnoreMarks;

        # Explicit 2 Parts

        sub a by a_ _a;
        sub o by o_ _o;
        sub e by a_ _e;
        sub p by a_ _p;
        sub y by o_ _y;
        sub c by c_ _c;
        sub s by s_ _s;
        
        # B
        
        sub bR by o_ _b;
        sub bL by b_ _o;

    } ETAPE_3_COMPOSANTS;

    lookup ETAPE_3_FUSION {  # Fusionne les composants des que possible

        lookupflag IgnoreMarks;

        # AO OA

        sub _a o_ by _ao_;
        sub _o a_ by _oa_;

        sub _oa_ a_ by _oa_;
        sub _ao_ o_ by _ao_;

        # S

        sub _s a_ by _sa_;
        sub _s b_ by _sb_;
        sub _s c_ by _sc_;
        
        sub _s uR by uR;

        # K

        sub k s_ by ks_;

        # T

        sub t s_ by ts_;
        sub t a_ by ta_;
        sub t o_ by to_;
        sub t b_ by tb_;
        sub t c_ by tc_;

    } ETAPE_3_FUSION;

    # ======================================================== 4 ESTHETIQUE

    lookup ETAPE_4_ESTHETIQUE {  # Applique les formes purement esthétiques

        lookupflag IgnoreMarks;
        
        # HORIZONTAL CONNECTORS

        sub [n fL] a_' by _na_;
        sub [n fL] c_' by _nc_;
        sub [n fH] o_' by _no_;
        sub [_a _p] b_' by _bn_;
        
        sub _c' [n fL] by _cn_;
        sub _a' [n fL] by _an_;
        sub _o' [n fH] by _on_;
        
        sub _p' [n fL] by _pn_;
        sub _e' [n fL s_] by _en_;
        sub _b' [n fH] by _bn_;

        # U FORMS

        sub kH uR' by uR_after_high;
        sub _s uR' by uR_after_high;

        sub n uL' by uL_after_n;
        sub [kH fH] uL' by uL_after_high;
        
        sub uR' n by uR_before_n;
        sub uR' [fH] by uR_before_high;

        sub t' uR by tuR_;

        # B ENDING FORMS

        sub [fL] b_' _o' [a_ o_ s_] by b_; # Ignoring certain combinations
        sub _e' b_' _o' by b_ending;
        sub _o o_' _b' by b_ending;  # _b implies it is not connected to the next letter
        # TODO : Mix b_ending to bs, bn, bf, bt, bk depending on the consonnant that follows
        # TODO : Ignore b_ending if followed by any other shit

        # S ENDING FORMS

        sub [_a n fH] s_' _s' by s_after_high;
        sub [fL _en_] s_' _s' by s_after_low;
        
        sub _o' s_' _s' by os_ending;
        sub _c' s_' _s' by cs_ending;
        sub _p' s_' _s' by ps_ending;
        sub _y' s_' _s' by ys_ending;

    } ETAPE_4_ESTHETIQUE;

    # ======================================================== 5 ESTHETIQUE DIACRITIQUE
    
    lookup ETAPE_5_DESTHETIQUE {  # Applique les formes purement esthétiques aux diacritiques
    
    sub i_it' by i_it;
    
    } ETAPE_5_DESTHETIQUE;

    # ======================================================== 6 KERNING

    lookup ETAPE_6_KERNING {

        lookupflag IgnoreMarks;

        # MERGE VOYELLE -80

        pos [_a _o] [a_ o_] -80;
        pos _o b_ending -5;

        # VOYELLE U

        pos tuR_ [uR uR_before_n uR_before_high] -160;
        pos s_ [uR uR_before_n uR_before_high] -120;

        # MERGE VOYELLE S -80

        pos [_s] [o_] -80;
        pos _a s_after_high -80;
        pos [fL _en_] s_after_low -80;

        pos [n fH] s_after_high -5;

        # MERGE VOYELLE F N

        pos [_a _o _p _an_ _on_ _cn_ _pn_ _bn_ _en_] [fH fL n _bn_] -20;
        pos [fH fL n] [a_ o_ _na_ _no_ _nc_] -20;

        # POSITIONNEMENT K T VOYELLE

        pos kH c_ -40;

        # MERGE K T S

    } ETAPE_6_KERNING;
    
    # ======================================================== 7 DIACRITIC KERNING

    lookup ETAPE_7_DKERNING {

        pos [i_ja] <0 800 0 00>;  # x-advance y-advance x-placement y-placement (placement moves the glyph)

    } ETAPE_7_DKERNING;

    # ======================================================== 7 

} ccmp;
'''

In [76]:
try: counter += 1
except: counter = 0
if not os.getcwd().endswith('bin'): 
    notebook_dir = os.getcwd()  # Actual working directory
    %cd ".\spetekkimyo\ffpython\bin\"
json.dump(padding, open(f'{notebook_dir}\\spetekkimyo\\input\\padding.json', 'w'))
with open(f'{notebook_dir}\\spetekkimyo\\input\\features.fea', 'w') as f: f.write(fea)
!"ffpython.exe" "..\..\generate.py" "{notebook_dir}\output\test.otf"

Imported 86 glyphs: a a_ b bL bR b_ b_ending c cs_ending c_ e f fH fL h h_ah h_ha i i_aj i_ia i_it i_ja i_sia i_tia i_tiu j k kH kL ks ks_ n o os_ending o_ p ps_ending s space s_ s_after_high s_after_low t ta_ tb_ tc_ tks_ tk_high tk_low to_ ts_ tuR_ u uL uL_after_high uL_after_n uR uR_after_high uR_before_high uR_before_n y ys_ending _a _an_ _ao_ _b _bn_ _c _cn_ _e _en_ _fc _na_ _nc_ _no_ _o _oa_ _on_ _p _pn_ _s _sa_ _sb_ _sc_ _y _yn_
Imported features
Font generated at d:\Github\spetekkimyo\output\test.otf


I'm sorry this file is too complex for me to understand (or is erroneous, or is empty)


In [77]:
e = """
<span class="ss">injajajojaja ja sja sai ha ah</span> pb
"""

In [78]:
html = """<style>p {font-size: 100px;} @font-face {src:url('./output/test.otf?version=%s');font-family:'test';} .ss {font-family:'test';}</style>
<p>%s</p>""" % (counter, e)
display(HTML(html))

### Guidelines

Epaisseur du trait : 80

Largeur : **550** (275) -- consonants 320 - 440 (max d'épaisseur) - 550 (pointe droite)

Hauteur standard : 320 -- max = 730 -- low = -240

Départ : h=-12